## Домашняя работа 5

**Задание простого уровня** Мы говорили, что метрики качества нужны, чтобы сравнивать различные модели между собой. В задаче полиномиальной регрессии реализуйте код для выбора лучшей степени полиному:

* возьмите все степени от 1 до 10 по порядку, без пропусков.
* найдите степень полинома будет лучший r2-score
* напишите код, который выводит самую подходящую степень полинома и соответствующий ей скор

Эта процедура называется Grid Search и помогает найти лучшие параметры для модели.

Обучите лучшую модель и сделайте predict

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
%matplotlib inline
df=pd.read_csv('non_linear.csv')

def x_degree(x: list, degree: int):
    return np.array([x**d for d in range(1, degree+1)]).T

def polinom_regr(features:list, result:list, degree: int):
    X=x_degree(features, degree)
    R=LinearRegression().fit(X, df.y_train)
    y_pred=R.predict(X)
    return r2_score(y_pred,result)

def best_polinom(features:list, result:list, degree: int):
        scores=[polinom_regr(features, result, i) for i in range(1,degree+1)]
        print (f'Лучшая степень полинома - {scores.index(max(scores))+1}, R2 = {max(scores)}')  

best_polinom(df['x_train'], df.y_train, 10)      

Лучшая степень полинома - 10, R2 = 0.9000272159538963


**Задание среднего уровня** Напишите класс для обучения модели, который содержит:

* функцию `.fit(X, y)` , которая принимает на вход массив фичей `X`, массив таргетов `y` и обучает коэффициенты регрессии. Код для обучения взять из первого урока модуля *Постановка ML задачи линейной регрессии*
* функцию `.predict(X)`, которая по массиву фичей `X` возвращает массив предсказаний `y`

Нужно использовать код для аналитически вычисяемых коэффициентов. 

Это задание позволит понять, как работает линейная регрессия "внутри" библиотечной реализации.

In [8]:
class CustomLinearReg:
    from numpy.linalg import inv
    def __init__(self):
        pass
    
    def fit(self, X, y):
        X_T_X = (X.T).dot(X)
        X_T_X_inverted = inv(X_T_X)
        self.w = X_T_X_inverted.dot(X.T).dot(y)
        return self
    def predict(self, X):
        return (self.w[0]+X*self.w[1])

In [9]:
X = np.array([[1, 50],[1, 60],[1, 70],[1, 100]])
x = np.array([[50],[60],[70],[100]])
y = np.array([[10],[30],[40],[50]])

In [11]:
model=CustomLinearReg().fit(X,y)
model.predict(x)

array([[18.21428571],
       [25.35714286],
       [32.5       ],
       [53.92857143]])

**Задание высокого уровня**

1. разделите датасет с домами Бостона из Урока 2 (таргет и фичи) на две части: в одной части 80% датасета (назовём train) в другой 20% (назовём valid)
1. обучите модель только на train датасете
1. постройте предсказания valid датасете
1. Посчитайте  `r2 score` на валидационном сете

После этого примените к обеим датасетам z-преобразование и повторите шаги 2-4. Как изменилась метрика r2?

Это задание поможет понять, как валидировать линейную регрессию (и другие модели) на отложенной выборке.

In [97]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
boston_dataset = load_boston()

features = boston_dataset.data
y = boston_dataset.target
x_train, x_valid, y_train, y_valid = train_test_split(features, y, test_size=0.2)

In [98]:
train=LinearRegression().fit(x_train, y_train)
valid=train.predict(x_valid)
r2_score(valid,y_valid)

0.6641811266757989

In [113]:
features_tr=StandardScaler().fit_transform(features)
y_tr=StandardScaler().fit_transform(y.reshape(-1,1)).reshape(-1)
x_train_tr, x_valid_tr, y_train_tr, y_valid_tr = train_test_split(features_tr, y, test_size=0.2)
train_tr=LinearRegression().fit(x_train_tr, y_train_tr)
valid_tr=train.predict(x_valid_tr)
r2_score(valid_tr,y_valid_tr)

-0.11604093288593664

In [114]:
features_tr=StandardScaler().fit_transform(features)
y_tr=StandardScaler().fit_transform(y.reshape(-1,1)).reshape(-1)
x_train_tr, x_valid_tr, y_train_tr, y_valid_tr = train_test_split(features_tr, y_tr, test_size=0.2)
train_tr=LinearRegression().fit(x_train_tr, y_train_tr)
valid_tr=train.predict(x_valid_tr)
r2_score(valid_tr,y_valid_tr)

-4.203400651032664

После применения z-преобразования метрика сильно ухудшилась. При обычных данных r2_score=0.66, после преобразования всего датасета получается -4.20, если преобразовать только фичи, то -0.11